# Numerical Optimization using JuMP


# JuMP

[JuMP](https://github.com/JuliaOpt/JuMP.jl) is a Julia package that provides a modeling language for general optimization problems.

## Optimization in Julia

Julia's optimization packages have become popular due to their ease of use, and variety of interfaces to mature solvers.  The main optimization functionality in Julia is provided by [JuliaOpt](http://www.juliaopt.org/).

The two high-level interfaces that you are most likely to use are
* [JuMP](https://github.com/JuliaOpt/JuMP.jl) - a modeling language for all sorts of optimization problems
* [Convex.jl](https://github.com/JuliaOpt/Convex.jl) - a package for disciplined convex programming (like [CVX](http://cvxr.com/cvx/))

There is a mid-level interface as well, [MathProgBase.jl](https://github.com/JuliaOpt/MathProgBase.jl).

The powerful thing about all of the above is that it is largely **solver independent**. This means you can forumlate the optimization problem with these packages, and then choose from a variety of solvers to use under the hood.  This is just like other modeling languages like AMPL - the reason why Julia's optimization packages have become popular is that they are generally easier to use than older modeling languages.

## Solvers

Today is more about turning your optimization models into something that can run on a computer via JuMP, but you still need a solver to actually solve the problem for you under the hood.  [JuliaOpt](http://www.juliaopt.org/) has a list of solvers that can be called from the high level interfaces (there are currently 20).  There are many open-source options available, but there are also interfaces to some of the big commercial solvers such as [Gurobi](http://www.gurobi.com/), [Mosek](https://www.mosek.com/), [Knitro](https://www.artelys.com/en/optimization-tools/knitro), etc. Many of these commercial solvers offer free academic/student licences, and if you are trying to solve large optimization problems they may be worth looking at.

# Last Class

Last class, we saw JuMP for the first time.  We covered linear programs (LPs), quadratic programs (QPs), second order cone constraints, and semi-definite programs (SDPs).  All of these problems arise have important applications in science and engineering.

# Today

We're going to cover several other types of optimization problems:
* Integer Programs (IPs)
* Mixed Integer Programs (MIPs)
* Nonlinear Programs (NLPs)

# Integer Programming

[Integer Programs (IPs)](https://en.wikipedia.org/wiki/Integer_programming) add integer constraints to the optimization variables.  This makes sense when variables come in unit quantities (e.g. people).  These problems take the form
\begin{align*}
\underset{x}{\text{minimize}} &~f(x)\\
\text{subject to:} &~x\in \mathbb{Z}\\
&c_e(x) = 0\\
&c_i(x) \le 0\\
\end{align*}
In JuMP, you can specify if a variable is an integer inside the variable declaration:

In [ ]:
using JuMP, Cbc
m = Model(solver=CbcSolver())
@variable(m, x >= 0, Int) # Int keyword says that x should be an integer
@variable(m, y >= 0, Int)

@objective(m, Max, x + y)
@constraint(m, 2*x + 2*y <= 5)
m

In [ ]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

there is also a `Bin` keyword that constrains a variable to be in {0,1}.

This is good for variables that denote if something exists or doesn't exist.

In [ ]:
using JuMP, Cbc
m = Model(solver=CbcSolver())
@variable(m, x >= 0, Bin) # Int keyword says that x should be an integer
@variable(m, y >= 0, Int)

@objective(m, Max, 2*x + y)
@constraint(m, 2*x + 2*y <= 7)
m

In [ ]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

# Mixed Integer Programming

Mixed Integer Programs (MIPs) have some variables that are constrained to be integers, and some that are not.

callbacks:

http://www.juliaopt.org/JuMP.jl/0.18/callbacks.html

In [ ]:
using JuMP, Cbc
m = Model(solver=CbcSolver())
@variable(m, x >= 0, Int) # Int keyword says that x should be an integer
@variable(m, y >= 0)

@objective(m, Max, x + y)
@constraint(m, 2*x + 2*y <= 5)
m

In [ ]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

# Solver Callbacks

Some optimzation solvers allow you to modify the problem in some way while the solver runs using callbacks.  This is covered in JuMP's documentation [here](http://www.juliaopt.org/JuMP.jl/0.18/callbacks.html).  This isn't supported by all solvers, and those that do tend to be commercial solvers ([see the solver table here](http://www.juliaopt.org/)).

We'll demo this using [Gurobi](https://github.com/JuliaOpt/Gurobi.jl).

In [ ]:
# We will use Gurobi
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

# very large bounds on x,y
@variable(m, -10 <= x <= 10, Int)
@variable(m, -10 <= y <= 10, Int)

@objective(m, Max, x^2+y^2 - y)


# L1 ball
# |x| + |y| <= 3
function l1_ball(cb)
    x_val = getvalue(x)
    y_val = getvalue(y)
    sx = sign(x_val)
    sy = sign(y_val)
    ax = sx * x_val # |x|
    ay = sy * y_val # |y|
    
    TOL = 1e-6
    l1r = 3 # L1 ball radius
    
    # add lazy constraint if we're outside of L1 ball by at least TOL
    if ax + ay > l1r + TOL
       @lazyconstraint(cb, sx*x + sy*y <= l1r) 
    end
end # end callback fn

addlazycallback(m, l1_ball)
m

In [ ]:
solve(m)
# Print our final solution
println("Final solution: [ $(getvalue(x)), $(getvalue(y)) ]")

# Exercise - NP-complete problems

[NP-complete](https://en.wikipedia.org/wiki/NP-completeness) problems are decision problems that may have no polynomial-time algorithm to compute (if $P\ne NP$), although solutions can be verified in polynomial time.  They arise in computer science, operations research, and have a variety of applications.

Many of these problems can be formulated as optimization problems with integer constraints.

* Minimum Vertex Cover https://en.wikipedia.org/wiki/Vertex_cover
* Maximum Coverage https://en.wikipedia.org/wiki/Maximum_coverage_problem
* Max-Cut (Need commercial solver for QIP)

While finding an optimal solution may be difficult, in practice we can find good results with IPs.

## Maximum-Coverage

The Maximum-Coverage problem is as follows: given $m$ sets, and an integer $k$, maximize the number of elements covered by at most $k$ sets. We'll assume that each of the $m$ sets is a subset of $N$ elements.

An application to think of: you want to buy $N$ items.  There are $m$ stores you could possibly go to, and each store only carries some of the items you'd like to buy.  Unfortunately you only have time to go to $k$ of the stores, but you'd like to get as much of your shopping done as possible (maximize the number of items you get).  Which stores should you go to today?

In [ ]:
# generates a Maximum-Coverage problem
# INPUTS:
# N = Number of items
# m = number of sets
# nsamples = number of sets for each sample
# OUTPUTS:
# S - Vector of Vectors
# S[i] = array of sets that contain ith element
function gen_max_cover(N, m, nsamples=2)

    # create assign elements to the nS sets
    S = Vector{Vector{Int64}}(N)
    
    for i = 1:N
       S[i] = unique(sample(1:m, nsamples))
    end
    return S
end
;

In [ ]:
# sets up a max-cover problem
k = 2 # number of sets

N = 10
nS = 5
S = gen_max_cover(N, nS)
;

### Step 1

Choose $k$ sets at random.  How many items are covered?  Try running a few trials

In [ ]:
# Your code here

### Step 2

Write an integer program using JuMP.  How many items are covered?  Try running a few trials

In [ ]:
# Your code here

# Non-linear Programming

JuMP also allows you to model more general [nonlinear problems (NLPs)](https://en.wikipedia.org/wiki/Nonlinear_programming).  To do this, you will want to use the macros `@NLobjective`, and `@NLconstraint`.  These can be combined with objectives and constraints we've already seen.

* JuMP's Introduction to solving NLPs [here](http://www.juliaopt.org/JuMP.jl/0.18/nlp.html)

## Example

Here, we'll look at the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function)
$$
f(x,y) = (a-x)^2 + b(y-x^2)^2
$$
For $b>0$, this function has a global minimum at $(x,y) = (a,a^2)$, where $f(x,y) = 0$.

In [ ]:
using Plots
plotlyjs()
a = 1.0
b = 50.0
f(x,y) = (a - x)^2 + b*(y - x^2)^2
n = 100
xs = ones(n)*linspace(-1.5,1.5,n)'
ys = linspace(-1.5,1.5,n)*ones(n)'
fs = f.(xs, ys)
surface(xs, ys, fs)

In [ ]:
# From http://www.juliaopt.org/JuMP.jl/0.18/nlp.html
using JuMP, Ipopt
m = Model(solver=IpoptSolver())
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

## Automatic Differentiation

Most optimization solvers need gradients and hessians in order to work.  How does JuMP obtain this information?

The answer is that all this information is obtained through [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) which uses the rules of differential calculus to differentiate functions just like you would.  Note that **this is different from using finite difference schemes** and is typically much more accurate.

The packages that JuMP uses for automatic differentiation are
* [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl)
* [Calculus.jl](https://github.com/JuliaMath/Calculus.jl)

You can also use these packages for your own purposes outside of JuMP.  As long as a function is built from core functions (e.g. you aren't calling special function libraries), products and sums, you can differentiate it.

In [ ]:
using ForwardDiff

f(x::Vector) = sum(x)
n = 5
x = randn(5)
f(x)

In [ ]:
ForwardDiff.gradient(f,x)

In [ ]:
ForwardDiff.hessian(f,x)

You can also wrap the gradient and hessian functions

In [ ]:
g(x) = ForwardDiff.gradient(f,x)
g(x)

In [ ]:
# an example on a more complicated function
f(x::Vector) = x[1]*x[2] + sum(sin.(x[3:end]))
g(x) = ForwardDiff.gradient(f,x)
g(x)

In [ ]:
@time f(x)

## Using Custom Functions for NLP in JuMP

Above, we wrote the Rosenbrock function explicitly as the objective function
```julia
@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)
```
We can also provide a function wrapper for the function - all we need to do is "register" the function in JuMP.  You can find informaiton in [JuMP's documentation here](http://www.juliaopt.org/JuMP.jl/0.18/nlp.html#user-defined-functions).

In [ ]:
using JuMP, Ipopt

rosenbrock(x,y) = (a - x)^2 + b*(y - x^2)^2

m = Model(solver=IpoptSolver(print_level=0))

# registers function with JuMP, and derivatives are computed
JuMP.register(m, :rosenbrock, 2, rosenbrock, autodiff=true)

@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, rosenbrock(x,y))

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

Let's break down the register command:
```julia
JuMP.register(m, :rosenbrock, 2, rosenbrock, autodiff=true)
```
| argument | description |
| ------ | -------- |
| `m` | model |
| `:rosenbrock` | Symbol used to identify funciton in objective|
| `2` | number of scalar inputs |
| `rosenbrock` | function we've declared |

If you can't forward diff your package, or you have an optimized gradient, you can also pass those in explicitly.  Note that JuMP currently doesn't support Hessians for functions of more than one variable.

In [ ]:
f(x,y) = x^2 + y^2
function ∇f(g,x,y) 
    g[1] = 2*x
    g[2] = 2*y
end

using JuMP, Ipopt

m = Model(solver=IpoptSolver())

# registers function with JuMP, and derivatives are computed
JuMP.register(m, :f, 2, f, ∇f)

@variable(m, x, start = 1.0)
@variable(m, y, start = 1.0)

@NLobjective(m, Min, f(x,y))

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))


## Nonlinear Constraints

You can specify noninear constraints using the `@NLconstraint` macro

In [ ]:
using JuMP, Ipopt

m = Model(solver=IpoptSolver())

@variable(m, x, start = 1.0)
@variable(m, y, start = 1.0)

@NLobjective(m, Min, (x-2)^4 + y^2)

@NLconstraint(m, x^2 - y == 0)

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

# Exercise

Finding the largest eigenvalue of a symmetric matrix can be formulated as an optimization problem

\begin{align*}
\text{maximize} &~x^T A x\\
\text{subject to:} &~ \|x\|_2^2 = 1
\end{align*}

The constraint $\|x\|_2^2 = x^Tx = 1$ is non-linear (and non-convex).  Find the largest eigenpair of `A = Diagonal([2; 1])` using JuMP.

Try this on a larger (symmetric) matrix.  How long does this take compared to `eig`, or `eigs`?

To time an operation, use Julia's `@time` macro, e.g. `@time solve(m)`

In [ ]:
# your code here

# Exercise

Find a local optimum for the folowing optimization problem

\begin{align*}
\underset{x,y,z}{\text{minimize}} &~x^3 + y^2 - x^4 + y + z\\
\text{subject to:} &~x^2 + y^2 + z^2 \le 10\\
&z^3 > 2
\end{align*}

In [ ]:
# your code here

# Exercise

We'll try finding an optimal point on a [lemniscate](https://en.wikipedia.org/wiki/Lemniscate):

\begin{align*}
\underset{x,y}{\text{maximize}} &~ x^3 + y^3\\
\text{subject to:} &~ x^4 - x^2 + y^2 = 0
\end{align*}

In [ ]:
# your code here

# HW 4

There's a short HW 4 posted in the [hw folder](../../hw/hw4/hw4.md).  Feel free to start in-class.

# Extras

* We used the Ipopt solver in our demo NLPs.  Check out [NLopt.jl](https://github.com/JuliaOpt/NLopt.jl) for another (free/open) option.  This is an itnterface to the [NLopt](https://nlopt.readthedocs.io/en/latest/) library. 
* Check out [JuMP's examples](http://www.juliaopt.org/notebooks/) for some interesting applied problems, such as rocket trajectories and solving sudoku